In [ ]:


!nvidia-smi -L


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:


!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-05-20 13:40:01--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-20 13:40:01 (96.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:


!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

unzip_data("nlp_getting_started.zip")

--2024-05-20 13:40:02--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.207, 142.250.98.207, 142.250.97.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-05-20 13:40:03 (106 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:


import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sample_submission_df = pd.read_csv("sample_submission.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:

sample_submission_df.head()


,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:


import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")



Target: 1 (real disaster)
Text:
People are more worried about the burning of buildings than black people losing their lives disgusting.

---

Target: 1 (real disaster)
Text:
Bradford. Back to doing what we do best. Burning down our own buildings. Read it and weep Leeds. https://t.co/OLnfzb86zb

---

Target: 1 (real disaster)
Text:
@EPCOTExplorer my jealous tears are burning with the fire of a thousand ransacked buildings. SO AWESOMEEEEEEEE

---

Target: 1 (real disaster)
Text:
@zourryart I forgot to add the burning buildings and screaming babies

---

Target: 1 (real disaster)
Text:
Sinking ships burning buildings &amp; Falling objects are what reminds me of the old us.

---



In [ ]:

from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                            train_df["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)


In [ ]:


train_sentences[:10] , train_labels[:10]

(array(["'McFadden Reportedly to Test Hamstring Thursday' via @TeamStream http://t.co/jWq4KvJH2j",
        'w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3',
        "When I was cooking earlier I got electrocuted some crucial ?????? now I'm psychic lol",
        "I'm On Fire.  http://t.co/WATsmxYTVa",
        "More than 40 families affected by the fatal outbreak of Legionnaires' disease in Edinburgh are to sue two comp... http://t.co/vsoXioOy78",
        'Why do u ruin everything?  @9tarbox u ruined the sour cream and u put a brick of cheese in the freezer..dummy',
        'http://t.co/c1H7JECFrV @RoyalCarribean do your passengers know about the mass murder that takes place in the #FaroeIslands every year?',
        "WACKOES like #MicheleBachman predict the WORLD will SOON be OBLITERATED by a burning firey INFERNO but can't accept #GlobalWarming!! HELLO!!",
        '@Reuters people like you should be charged after the inevitable rioting for contributing to

In [ ]:

train_sentences[:5]


array(["'McFadden Reportedly to Test Hamstring Thursday' via @TeamStream http://t.co/jWq4KvJH2j",
       'w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3',
       "When I was cooking earlier I got electrocuted some crucial ?????? now I'm psychic lol",
       "I'm On Fire.  http://t.co/WATsmxYTVa",
       "More than 40 families affected by the fatal outbreak of Legionnaires' disease in Edinburgh are to sue two comp... http://t.co/vsoXioOy78"],
      dtype=object)

In [ ]:


import tensorflow as tf

from tensorflow.keras.layers import TextVectorization


text_vectorizer = TextVectorization(max_tokens = 10000,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None,
    pad_to_max_tokens = True)

In [ ]:


text_vectorizer = TextVectorization(max_tokens = 10000,
     output_mode = "int",
     output_sequence_length = 15)





In [ ]:

text_vectorizer.adapt(train_sentences)



In [ ]:

random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])





Original text:
RT @HuffPostComedy: We should build a wall that keeps Burning Man attendees from coming home http://t.co/xwVW1sft4I http://t.co/j7HUKhWmal      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 105,    1,   47,  149, 1430,    3, 1228,   16, 1513,   84,   96,
           1,   20,  233,  147]])>

In [ ]:


words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 most common words: {bottom_5_words}")



Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 most common words: ['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']


In [ ]:



from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = 10000,
                             output_dim = 128,
                             input_length = 15)
embedding

In [ ]:

random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed


Original text:
Mini Lalaloopsy Dolls Seed Sunburst June Seashore Sweater Snowstorm Autumn Spice - Full reÛ_ http://t.co/nyty7fCQo6 http://t.co/hyypsPN0yQ      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03785905,  0.00326221,  0.02713436, ..., -0.03529821,
          0.00196572, -0.02653354],
        [-0.04303893, -0.00485375,  0.04376778, ..., -0.02873641,
         -0.03268979,  0.04453868],
        [-0.04303893, -0.00485375,  0.04376778, ..., -0.02873641,
         -0.03268979,  0.04453868],
        ...,
        [ 0.02230254, -0.03252157, -0.04361831, ...,  0.03933748,
         -0.02758133, -0.00708713],
        [-0.04303893, -0.00485375,  0.04376778, ..., -0.02873641,
         -0.03268979,  0.04453868],
        [-0.04303893, -0.00485375,  0.04376778, ..., -0.02873641,
         -0.03268979,  0.04453868]]], dtype=float32)>

In [ ]:

sample_embed[0][0], sample_embed[0][0].shape , random_sentence



(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03785905,  0.00326221,  0.02713436, -0.02688417, -0.03066148,
         0.04912266, -0.01729639, -0.04906466, -0.04292297,  0.0290865 ,
         0.0498557 , -0.04494852, -0.01057504, -0.00770976,  0.04659257,
         0.02591285, -0.02845161,  0.02341778, -0.02895321,  0.00444593,
        -0.01154935, -0.01020138, -0.02228681,  0.04561972, -0.00013313,
         0.04845686,  0.03926994, -0.03586287, -0.04680076,  0.03734681,
        -0.04582984, -0.01006397,  0.00661432, -0.04103495, -0.03263567,
         0.03569026, -0.01070148, -0.04640696,  0.04103812, -0.04618826,
        -0.0362288 , -0.04167631,  0.01461213,  0.04009548,  0.00608724,
        -0.0282698 , -0.02731865, -0.03652986, -0.01724894,  0.04868645,
         0.03161972,  0.02596159, -0.00668601, -0.00121305,  0.00296246,
        -0.03768629,  0.04290924,  0.02009842,  0.02304738, -0.01636172,
        -0.00557948,  0.04047186, -0.04415014,  0.001042  , -0.03262801,
  


Model 0:Naive Bayes (baseline)

Model 1:Feed-forward neural network (dense model)

Model 2:LSTM model (RNN)

Model 3:GRU model (RNN)

Model 4:Bidirectional-LSTM model (RNN)

Model 5:1D Convolutioanl Neural Network (CNN)

Model 6:Tensorflow Hub Pretrained Feature Extractior (using transfer learning for NLP)

Model 7:Same as model 6 but with 10% of training data

Create

Build

Fit

Evaluate



In [ ]:
#Naive Bayes

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:

baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")



Our baseline model achieves an accuracy of: 77.82%


In [ ]:


baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [ ]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100

  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")

  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results





In [ ]:


baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 77.82152230971128,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129,
 'f1': 0.7703527809038113}

In [ ]:
#Feed-Forward Neural Network



from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [ ]:

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
#x = layers.MaxPooling1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")


In [ ]:

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [ ]:


model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])



In [ ]:


model_1_history = model_1.fit(x = train_sentences,
                      y = train_labels,
                      epochs =5,
                      validation_data = (val_sentences, val_labels),
                      callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                      experiment_name = "model_1_dense")])



Saving TensorBoard log files to: model_logs/model_1_dense/20240520-134005
Epoch 1/5
215/215 [==============================] - 7s 27ms/step - loss: 0.6111 - accuracy: 0.6967 - val_loss: 0.5416 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 6s 29ms/step - loss: 0.4396 - accuracy: 0.8205 - val_loss: 0.4848 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 6s 29ms/step - loss: 0.3454 - accuracy: 0.8621 - val_loss: 0.4765 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.2828 - accuracy: 0.8908 - val_loss: 0.4850 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.2362 - accuracy: 0.9113 - val_loss: 0.5002 - val_accuracy: 0.7913


In [ ]:

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.5002 - accuracy: 0.7913


[0.5002145767211914, 0.7913385629653931]

In [ ]:


model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [ ]:

model_1_pred_probs[:10]




array([[0.02834575],
       [0.14383475],
       [0.3511415 ],
       [0.01986792],
       [0.605962  ],
       [0.13988985],
       [0.02559577],
       [0.26020038],
       [0.10183968],
       [0.90946233]], dtype=float32)

In [ ]:


model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 0.], dtype=float32)>

In [ ]:


model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds)
model_1_results

{'accuracy': 79.13385826771653,
 'precision': 0.7928181179376032,
 'recall': 0.7913385826771654,
 'f1': 0.7889473553291934}

In [ ]:

baseline_results

{'accuracy': 77.82152230971128,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129,
 'f1': 0.7703527809038113}

In [ ]:


embed_weights = model_1.get_layer("embedding").get_weights()[0]

embed_weights

array([[ 0.0077412 , -0.04982882,  0.05924788, ...,  0.01807789,
        -0.00339821, -0.01589657],
       [-0.04942765, -0.01129219,  0.05054665, ..., -0.03484451,
        -0.03982468,  0.03850393],
       [-0.03790904,  0.00544043, -0.0124486 , ..., -0.03800693,
         0.01339581, -0.05079684],
       ...,
       [ 0.07915524,  0.0510242 , -0.04355108, ...,  0.03274063,
         0.10679732,  0.05098608],
       [-0.00403206,  0.01661125, -0.04441954, ..., -0.04238131,
        -0.0490423 , -0.00962248],
       [-0.00838904, -0.04397587,  0.04050307, ...,  0.02814821,
        -0.02496406,  0.02602966]], dtype=float32)

In [ ]:
embed_weights.shape


(10000, 128)

In [ ]:


words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab) , words_in_vocab[:10]




(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:

# import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [ ]:


# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

In [ ]:
#RNN (LSTM)

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(64, return_sequences = True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name = "model_2_LSTM")




In [ ]:


model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:

model_2.compile(loss= 'binary_crossentropy',
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])






In [ ]:


model_2_history = model_2.fit(x=train_sentences,
                      y=train_labels,
                      epochs=5,
                      validation_data=(val_sentences, val_labels),
                      callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                          experiment_name='model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240520-134035
Epoch 1/5
215/215 [==============================] - 14s 53ms/step - loss: 0.2180 - accuracy: 0.9203 - val_loss: 0.6320 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 9s 44ms/step - loss: 0.1532 - accuracy: 0.9425 - val_loss: 0.5686 - val_accuracy: 0.7598
Epoch 3/5
215/215 [==============================] - 7s 32ms/step - loss: 0.1246 - accuracy: 0.9550 - val_loss: 0.8040 - val_accuracy: 0.7441
Epoch 4/5
215/215 [==============================] - 9s 40ms/step - loss: 0.1029 - accuracy: 0.9618 - val_loss: 0.8865 - val_accuracy: 0.7402
Epoch 5/5
215/215 [==============================] - 8s 40ms/step - loss: 0.0794 - accuracy: 0.9666 - val_loss: 0.9729 - val_accuracy: 0.7349


In [ ]:

model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 8ms/step


array([[1.9468581e-03],
       [4.8405264e-02],
       [2.1661010e-03],
       [3.1481290e-04],
       [9.5501679e-01],
       [1.7470461e-01],
       [5.7774148e-04],
       [1.1842614e-02],
       [2.7115593e-02],
       [9.9921530e-01]], dtype=float32)

In [ ]:


model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=tf.squeeze(tf.round(model_2_pred_probs)))
model_2_results

{'accuracy': 73.49081364829397,
 'precision': 0.7356763794422214,
 'recall': 0.7349081364829396,
 'f1': 0.7352111408432463}

In [ ]:

baseline_results

{'accuracy': 77.82152230971128,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129,
 'f1': 0.7703527809038113}

In [ ]:

#RNN (GRU)

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.GRU(64, return_sequences = True)(x)
print(x.shape)
# x = layers.LSTM(42, return_sequences = True)(x)
# print(x.shape)
# x = layers.GRU(99)(x)
# print(x.shape)
# x = layers.Dense(64, activation = "relu")(x)
x = layers.GlobalAveragePooling1D()(x)

outputs = layers.Dense(1, activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name = "model_3_GRU")


(None, 15, 128)
(None, 15, 64)


In [ ]:

model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 global_average_pooling1d_1  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 

In [ ]:

model_3.compile(loss= 'binary_crossentropy',
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])

In [ ]:


model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                  experiment_name='model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240520-134125
Epoch 1/5
215/215 [==============================] - 11s 35ms/step - loss: 0.1748 - accuracy: 0.9494 - val_loss: 0.7041 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 9s 44ms/step - loss: 0.0984 - accuracy: 0.9635 - val_loss: 0.8694 - val_accuracy: 0.7467
Epoch 3/5
215/215 [==============================] - 9s 44ms/step - loss: 0.0805 - accuracy: 0.9705 - val_loss: 1.0119 - val_accuracy: 0.7388
Epoch 4/5
215/215 [==============================] - 9s 43ms/step - loss: 0.0717 - accuracy: 0.9723 - val_loss: 1.0499 - val_accuracy: 0.7441
Epoch 5/5
215/215 [==============================] - 8s 39ms/step - loss: 0.0623 - accuracy: 0.9729 - val_loss: 1.0370 - val_accuracy: 0.7388


In [ ]:

model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[3.2759674e-03],
       [1.2027178e-01],
       [7.3651737e-04],
       [2.1495380e-05],
       [8.5281390e-01],
       [5.7633951e-02],
       [8.7631688e-06],
       [3.1415481e-04],
       [4.5222067e-03],
       [9.9991733e-01]], dtype=float32)

In [ ]:


model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=tf.squeeze(tf.round(model_3_pred_probs)))
model_3_results

{'accuracy': 73.88451443569554,
 'precision': 0.7383815192323109,
 'recall': 0.7388451443569554,
 'f1': 0.7361920999405764}

In [ ]:

baseline_results

{'accuracy': 77.82152230971128,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129,
 'f1': 0.7703527809038113}

In [ ]:
# Bidirectional (LSTM) RNN

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

outputs = layers.Dense(1, activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_Bidirectional")




In [ ]:

model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 15, 128)           98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                             

In [ ]:


model_4.compile(loss= 'binary_crossentropy',
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])

In [ ]:

model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 31s 94ms/step - loss: 0.0956 - accuracy: 0.9658 - val_loss: 1.1752 - val_accuracy: 0.7388
Epoch 2/5
215/215 [==============================] - 18s 85ms/step - loss: 0.0509 - accuracy: 0.9784 - val_loss: 1.6518 - val_accuracy: 0.7192
Epoch 3/5
215/215 [==============================] - 16s 73ms/step - loss: 0.0475 - accuracy: 0.9787 - val_loss: 1.2540 - val_accuracy: 0.7493
Epoch 4/5
215/215 [==============================] - 17s 79ms/step - loss: 0.0406 - accuracy: 0.9803 - val_loss: 1.5398 - val_accuracy: 0.7507
Epoch 5/5
215/215 [==============================] - 19s 89ms/step - loss: 0.0446 - accuracy: 0.9777 - val_loss: 1.7472 - val_accuracy: 0.7520


In [ ]:

model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 5s 15ms/step


array([[3.5270970e-04],
       [2.9870365e-03],
       [2.5118102e-04],
       [5.1538063e-05],
       [9.9975717e-01],
       [5.7526823e-04],
       [7.0633476e-05],
       [1.2422052e-04],
       [2.5744273e-04],
       [9.9993372e-01]], dtype=float32)

In [ ]:


model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=tf.squeeze(tf.round(model_4_pred_probs)))
model_4_results

{'accuracy': 75.19685039370079,
 'precision': 0.7511278427687698,
 'recall': 0.7519685039370079,
 'f1': 0.7505865044907326}

In [ ]:

baseline_results

{'accuracy': 77.82152230971128,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129,
 'f1': 0.7703527809038113}

In [ ]:

#Conv1D


from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64, kernel_size =5, strides =1, activation = "relu", padding ="valid")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name = "model_5_Conv1D")

model_5.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_average_pooling1d_3  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_5 (Dense)             (None, 1)              

In [ ]:


model_5_history = model_5.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data=(val_sentences, val_labels),
                          callbacks = [create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240520-184816
Epoch 1/5
215/215 [==============================] - 11s 41ms/step - loss: 0.1522 - accuracy: 0.9539 - val_loss: 0.9626 - val_accuracy: 0.7428
Epoch 2/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0838 - accuracy: 0.9676 - val_loss: 1.1271 - val_accuracy: 0.7454
Epoch 3/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0656 - accuracy: 0.9726 - val_loss: 1.3210 - val_accuracy: 0.7402
Epoch 4/5
215/215 [==============================] - 6s 29ms/step - loss: 0.0566 - accuracy: 0.9774 - val_loss: 1.4885 - val_accuracy: 0.7323
Epoch 5/5
215/215 [==============================] - 8s 38ms/step - loss: 0.0511 - accuracy: 0.9777 - val_loss: 1.5956 - val_accuracy: 0.7165


In [ ]:

model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 8ms/step


array([[5.5788492e-05],
       [4.8382640e-01],
       [7.1009208e-06],
       [1.0943332e-04],
       [9.8982179e-01],
       [3.5641760e-01],
       [2.8792466e-09],
       [1.2496087e-03],
       [2.1270778e-02],
       [9.9999887e-01]], dtype=float32)

In [ ]:

model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=tf.squeeze(tf.round(model_5_pred_probs)))
model_5_results

{'accuracy': 71.65354330708661,
 'precision': 0.7201726378713678,
 'recall': 0.7165354330708661,
 'f1': 0.7173885788548277}

In [ ]:

baseline_results

{'accuracy': 77.82152230971128,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129,
 'f1': 0.7703527809038113}